In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import scipy
import numpy.ma as ma

In [95]:
n_features = [1, 5, 10, 20, 30, 40]

## APS Dataset

In [36]:
dataset_path_prefix = './Datasets/APS/Q1/'
APS_guest_full_test = pd.read_csv(dataset_path_prefix +  'APS_guest_full_test.csv', na_values = 'na', index_col = 'info')
APS_guest_full_train = pd.read_csv(dataset_path_prefix +  'APS_guest_full_train.csv', na_values = 'na', index_col = 'info')
APS_host_full_test = pd.read_csv(dataset_path_prefix +  'APS_host_full_test.csv', na_values = 'na', index_col = 'info')
APS_host_full_train = pd.read_csv(dataset_path_prefix +  'APS_host_full_train.csv', na_values = 'na', index_col = 'info')

In [51]:
# sanity check
assert(len(APS_guest_full_test) == len(APS_host_full_test))
assert(len(APS_guest_full_train) == len(APS_host_full_train))
assert(len(APS_guest_full_test.columns) == len(APS_host_full_test.columns)+1)
assert(len(APS_guest_full_train.columns) == len(APS_host_full_train.columns)+1)

86


In [55]:
APS_train_df = pd.concat([APS_guest_full_train, APS_host_full_train], axis = 1)
APS_test_df = pd.concat([APS_guest_full_test, APS_host_full_test], axis = 1)
print(len(APS_train_df.columns))

171


## coefficient of variance (statistic_filter)

In [108]:
def create_cv_dfs(df, n_features):
    cv_res = scipy.stats.variation(df, nan_policy = 'omit')
    cv_res = ma.getdata(cv_res).tolist()
    
    # remove the columns that coeficient of variance is 0
    to_rm = []
    for idx in range(len(cv_res)):
        if (cv_res[idx] == 0):
            to_rm.insert(0, idx)
    for idx in to_rm:
        print('removing column ' + str(idx) + ', because of 0 cv value')
        cv_res.pop(idx)
        APS_train_df_no_y.drop(APS_train_df_no_y.columns[idx] ,1)
        
    res = []
    # select features with n smallest cv value (larget than 0)
    for n_feature in n_features:
        smallest_idx = sorted(range(len(cv_res)), key=lambda k: cv_res[k])[:n_feature]
        print(smallest_idx)
        tmp = df.iloc[:, smallest_idx]
        print('df', len(tmp.columns), len(tmp))
        res.append(tmp)

In [109]:
APS_train_df_no_y = APS_train_df.drop('class', 1)
APS_train_y = APS_train_df['class']
create_cv_dfs(APS_train_df_no_y, n_features)

removing column 89, because of 0 cv value
[78]
df 1 6000
[78, 77, 87, 76, 71]
df 5 6000
[78, 77, 87, 76, 71, 75, 86, 74, 72, 73]
df 10 6000
[78, 77, 87, 76, 71, 75, 86, 74, 72, 73, 79, 112, 119, 130, 143, 124, 142, 118, 147, 2]
df 20 6000
[78, 77, 87, 76, 71, 75, 86, 74, 72, 73, 79, 112, 119, 130, 143, 124, 142, 118, 147, 2, 89, 22, 23, 146, 67, 16, 156, 62, 82, 110]
df 30 6000
[78, 77, 87, 76, 71, 75, 86, 74, 72, 73, 79, 112, 119, 130, 143, 124, 142, 118, 147, 2, 89, 22, 23, 146, 67, 16, 156, 62, 82, 110, 81, 103, 93, 12, 80, 0, 117, 159, 53, 84]
df 40 6000
